In [1]:
!pip3 install catboost

     |████████████████████████████████| 65.7MB 57kB/s 


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor

# from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
# from imblearn.pipeline import make_pipeline as imb_make_pipeline
# from imblearn.under_sampling import RandomUnderSampler


# from sklearn.preprocessing import RobustScaler, FunctionTransformer


# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_validate
# from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

# from sklearn.metrics import plot_confusion_matrix
# from sklearn.metrics import confusion_matrix

# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import auc, roc_auc_score, roc_curve
# from sklearn.metrics import recall_score, precision_score, f1_score, log_loss
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import precision_recall_curve
# from sklearn.metrics import classification_report
# from sklearn.metrics import mean_absolute_error

# from sklearn.preprocessing import PolynomialFeatures

import warnings
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.7.10 (default, Feb 20 2021, 21:17:23) 
Numpy        : 1.19.5


In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
# Убирает некритические сообщения
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [52]:
train = pd.read_csv('drive/My Drive/Colab Notebooks/Project_6/train.csv')
test = pd.read_csv('drive/My Drive/Colab Notebooks/Project_6/test.csv')
sample_submission = pd.read_csv('drive/My Drive/Colab Notebooks/Project_6/sample_submission.csv')

In [53]:
display(train.head(3))
display(test.head(3))

,name,brand,model_name,priceCurrency,description,productionDate,color,enginedesc,mileage,bodyTypeDesc,vehicleTransmission,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,sell_id,url_auto,fuelType
0,2.0 CVT (180 л.с.),Audi,A6,1 175 000 ₽,"Артикул: 26369ПРИ ПОКУПКЕ АВТОМОБИЛЯ В КРЕДИТ,...",2013,чёрный,2.0 л / 180 л.с. / Бензин,102 000 км,седан,вариатор,передний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,1102319418,https://auto.ru/cars/used/sale/audi/a6/1102319...,Бензин
1,3.0 AT (220 л.с.) 4WD,Audi,A6,360 000 ₽,Прoдaм ауди А6 в идeaльнoм состоянии.Чиcтокpов...,2003,фиолетовый,3.0 л / 220 л.с. / Бензин,231 200 км,седан,автоматическая,полный,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,1102661742,https://auto.ru/cars/used/sale/audi/a6/1102661...,Бензин
2,2.0 AT (225 л.с.) 4WD,Audi,Q5,2 150 000 ₽,Здравствуйте!А/м не требует вложений по кузову...,2014,белый,2.0 л / 225 л.с. / Бензин,67 082 км,внедорожник 5 дв.,автоматическая,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,1102577541,https://auto.ru/cars/used/sale/audi/q5/1102577...,Бензин


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,RUB,2014,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,RUB,2017,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,88000,2013,"{""code"":""SUPERB"",""name"":""Superb"",""ru_name"":""Су...",SUPERB,DSG 1.8 AMT (152 л.с.),5,1603226280,RUB,2014,1100658222,"{""id"":""20026323"",""nameplate"":""DSG"",""displaceme...",LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [ ]:
%%script false
display(pd.DataFrame(train['Владельцы'].value_counts()))
display(pd.DataFrame(test['Владельцы'].value_counts()))

In [54]:
#преобразуем mileage в число
train['mileage'] = train['mileage'].str.replace('\xa0', '')
train['mileage'] = train['mileage'].str.findall(r'\d{1,}').str[0]
train['mileage'] = train['mileage'].astype('int')

display(train.head(3))

#преобразуем priceCurrency в число price
train['priceCurrency'] = train['priceCurrency'].str.replace('\xa0', '')
train['price'] = train['priceCurrency'].str.findall(r'\d{1,}').str[0]
train['price'] = train['price'].astype('int')

display(train.head(3))

,name,brand,model_name,priceCurrency,description,productionDate,color,enginedesc,mileage,bodyTypeDesc,vehicleTransmission,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,sell_id,url_auto,fuelType
0,2.0 CVT (180 л.с.),Audi,A6,1 175 000 ₽,"Артикул: 26369ПРИ ПОКУПКЕ АВТОМОБИЛЯ В КРЕДИТ,...",2013,чёрный,2.0 л / 180 л.с. / Бензин,102000,седан,вариатор,передний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,1102319418,https://auto.ru/cars/used/sale/audi/a6/1102319...,Бензин
1,3.0 AT (220 л.с.) 4WD,Audi,A6,360 000 ₽,Прoдaм ауди А6 в идeaльнoм состоянии.Чиcтокpов...,2003,фиолетовый,3.0 л / 220 л.с. / Бензин,231200,седан,автоматическая,полный,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,1102661742,https://auto.ru/cars/used/sale/audi/a6/1102661...,Бензин
2,2.0 AT (225 л.с.) 4WD,Audi,Q5,2 150 000 ₽,Здравствуйте!А/м не требует вложений по кузову...,2014,белый,2.0 л / 225 л.с. / Бензин,67082,внедорожник 5 дв.,автоматическая,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,1102577541,https://auto.ru/cars/used/sale/audi/q5/1102577...,Бензин


,name,brand,model_name,priceCurrency,description,productionDate,color,enginedesc,mileage,bodyTypeDesc,vehicleTransmission,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,sell_id,url_auto,fuelType,price
0,2.0 CVT (180 л.с.),Audi,A6,1175000₽,"Артикул: 26369ПРИ ПОКУПКЕ АВТОМОБИЛЯ В КРЕДИТ,...",2013,чёрный,2.0 л / 180 л.с. / Бензин,102000,седан,вариатор,передний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,1102319418,https://auto.ru/cars/used/sale/audi/a6/1102319...,Бензин,1175000
1,3.0 AT (220 л.с.) 4WD,Audi,A6,360000₽,Прoдaм ауди А6 в идeaльнoм состоянии.Чиcтокpов...,2003,фиолетовый,3.0 л / 220 л.с. / Бензин,231200,седан,автоматическая,полный,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,1102661742,https://auto.ru/cars/used/sale/audi/a6/1102661...,Бензин,360000
2,2.0 AT (225 л.с.) 4WD,Audi,Q5,2150000₽,Здравствуйте!А/м не требует вложений по кузову...,2014,белый,2.0 л / 225 л.с. / Бензин,67082,внедорожник 5 дв.,автоматическая,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,1102577541,https://auto.ru/cars/used/sale/audi/q5/1102577...,Бензин,2150000


In [55]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''

    df_output = df_input.copy()

    #превратим владельцев в числа
    df_output['Владельцы'] = df_output['Владельцы'].apply(lambda x: x[0])
    df_output['Владельцы'] = df_output['Владельцы'].values.astype('int64')
    

    #вытащим показатели объема двигателя 'engineDisplacement' данные грязные из name
    df_output['engineDisplacement1'] = df_output['name'].str.findall(r'\d\.\d').str[0]
    df_output['engineDisplacement1'] = df_output['engineDisplacement1'].astype('str')



    #Заполним в Train engineDisplacement
    df_output['engineDisplacement'] = df_output['engineDisplacement1']
    df_output['engineDisplacement'] = df_output['engineDisplacement'].apply(lambda x: '2.0' if x == 'nan' else x)
    df_output['engineDisplacement'] = df_output['engineDisplacement'].astype('float32')
    df_output['engineDisplacement'] = df_output['engineDisplacement'].astype('int64')

    #так же вытащим показатели мощности двигателя 'enginePower'  из name
    df_output['enginePower'] = df_output['name'].str.findall(r'\d{2,3}').str[0]
    df_output['enginePower'] = df_output['enginePower'].astype('int64')


    # расчитаем возраст автомобиля
    df_output['years'] = 2021 - df_output['productionDate']

    #заполним пропуски в столбце description. Там где есть описание ставим 1 если нет то 0
    df_output['cd'] = (df_output['description'] == '0')
    df_output['description'] = df_output['cd'].map({False:1, True: 0})
    
    
    # ################### Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Таможня', 'Состояние', 'priceCurrency', 'productionDate', 'cd', 'engineDisplacement1'], axis=1, inplace=True,)
    

    return df_output

In [ ]:
%%script false
#df_output = df_input.copy()

#превратим владельцев в числа
train['Владельцы'] = train['Владельцы'].apply(lambda x: x[0])
train['Владельцы'] = train['Владельцы'].values.astype('int64')


#вытащим показатели объема двигателя 'engineDisplacement' данные грязные из name
train['engineDisplacement1'] = train['name'].str.findall(r'\d\.\d').str[0]
train['engineDisplacement1'] = train['engineDisplacement1'].astype('str')



#Заполним в Train engineDisplacement
train['engineDisplacement'] = train['engineDisplacement1']
train['engineDisplacement'] = train['engineDisplacement'].apply(lambda x: '2.0' if x == 'nan' else x)
train['engineDisplacement'] = train['engineDisplacement'].astype('float32')
train['engineDisplacement'] = train['engineDisplacement'].astype('int64')

#так же вытащим показатели мощности двигателя 'enginePower'  из name
train['enginePower'] = train['name'].str.findall(r'\d{2,3}').str[0]
train['enginePower'] = train['enginePower'].astype('str')

# всё забрали - удаляем name
#test.drop('name', axis=1, inplace=True)
#train.drop('name', axis=1, inplace=True)



# расчитаем возраст автомобиля
train['years'] = 2021 - train['productionDate']




# сам productionDate можно удалить: новый признак его заменяет полностью
#test.drop('productionDate', axis=1, inplace=True)
#train.drop('productionDate', axis=1, inplace=True)

#заполним пропуски в столбце description. Там где есть описание ставим 1 если нет то 0
train['cd'] = (train['description'] == '0')
train['description'] = train['cd'].map({False:1, True: 0})


# test = test.drop(['Состояние', 'Таможня', 'priceCurrency', 'car_url', 'complectation_dict',
#        'description', 'equipment_dict', 'image', 'modelDate', 'model_info', 
#        'parsing_unixtime', 'sell_id', 'super_gen', 'vehicleConfiguration',
#        'vendor', 'Владение'], axis=1)
# display(test.head(3))


#df_output = df_input.copy()

# ################### Предобработка ############################################################## 
# убираем не нужные для модели признаки
train.drop(['Таможня', 'Состояние', 'priceCurrency', 'productionDate', 'cd'], axis=1, inplace=True,)

In [ ]:
%%script false
display(pd.DataFrame(train['enginePower'].value_counts()))
unique, counts = np.unique(train.enginePower, return_counts=True)
dict(zip(unique, counts))

In [ ]:
%%script false
#display(train[train['enginePower']=='nan'])
train['name'].iloc[19183]

In [56]:
train_preproc = preproc_data(train)
X_sub = preproc_data(test)

In [57]:
display(train_preproc.head(3))
display(X_sub.head(3))

,name,brand,model_name,description,color,enginedesc,mileage,bodyTypeDesc,vehicleTransmission,Привод,Руль,Владельцы,ПТС,sell_id,url_auto,fuelType,price,engineDisplacement,enginePower,years
0,2.0 CVT (180 л.с.),Audi,A6,1,чёрный,2.0 л / 180 л.с. / Бензин,102000,седан,вариатор,передний,Левый,1,Оригинал,1102319418,https://auto.ru/cars/used/sale/audi/a6/1102319...,Бензин,1175000,2,180,8
1,3.0 AT (220 л.с.) 4WD,Audi,A6,1,фиолетовый,3.0 л / 220 л.с. / Бензин,231200,седан,автоматическая,полный,Левый,3,Дубликат,1102661742,https://auto.ru/cars/used/sale/audi/a6/1102661...,Бензин,360000,3,220,18
2,2.0 AT (225 л.с.) 4WD,Audi,Q5,1,белый,2.0 л / 225 л.с. / Бензин,67082,внедорожник 5 дв.,автоматическая,полный,Левый,1,Оригинал,1102577541,https://auto.ru/cars/used/sale/audi/q5/1102577...,Бензин,2150000,2,225,7


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,years
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,1,1,105,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3,NaN,Оригинал,передний,Левый,7
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,1,1,110,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1,NaN,Оригинал,передний,Левый,4
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...",1,1,152,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,88000,2013,"{""code"":""SUPERB"",""name"":""Superb"",""ru_name"":""Су...",SUPERB,DSG 1.8 AMT (152 л.с.),5,1603226280,1100658222,"{""id"":""20026323"",""nameplate"":""DSG"",""displaceme...",LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1,NaN,Оригинал,передний,Левый,7


In [58]:
X_sub.loc[(X_sub['ПТС'].isnull()), 'ПТС'] = 'Оригинал'
X_sub = X_sub.drop(['car_url', 'complectation_dict',
       'equipment_dict', 'image', 'modelDate', 'model_info', 'numberOfDoors', 
       'parsing_unixtime', 'super_gen', 'vehicleConfiguration',
       'vendor', 'Владение'], axis=1)
train_preproc = train_preproc.drop(['enginedesc', 'url_auto'], axis=1)
train_preproc.rename(columns = {'bodyTypeDesc':'bodyType'}, inplace = True)
display(X_sub.head(3))
display(train_preproc.head(3))

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,model_name,name,sell_id,vehicleTransmission,Владельцы,ПТС,Привод,Руль,years
0,лифтбек,SKODA,синий,1,1,105,бензин,74000,OCTAVIA,1.2 AMT (105 л.с.),1100575026,роботизированная,3,Оригинал,передний,Левый,7
1,лифтбек,SKODA,чёрный,1,1,110,бензин,60563,OCTAVIA,1.6 MT (110 л.с.),1100549428,механическая,1,Оригинал,передний,Левый,4
2,лифтбек,SKODA,серый,1,1,152,бензин,88000,SUPERB,DSG 1.8 AMT (152 л.с.),1100658222,роботизированная,1,Оригинал,передний,Левый,7


,name,brand,model_name,description,color,mileage,bodyType,vehicleTransmission,Привод,Руль,Владельцы,ПТС,sell_id,fuelType,price,engineDisplacement,enginePower,years
0,2.0 CVT (180 л.с.),Audi,A6,1,чёрный,102000,седан,вариатор,передний,Левый,1,Оригинал,1102319418,Бензин,1175000,2,180,8
1,3.0 AT (220 л.с.) 4WD,Audi,A6,1,фиолетовый,231200,седан,автоматическая,полный,Левый,3,Дубликат,1102661742,Бензин,360000,3,220,18
2,2.0 AT (225 л.с.) 4WD,Audi,Q5,1,белый,67082,внедорожник 5 дв.,автоматическая,полный,Левый,1,Оригинал,1102577541,Бензин,2150000,2,225,7


In [61]:
cols = list(X_sub.columns.values)
print(cols)


['bodyType', 'brand', 'color', 'description', 'engineDisplacement', 'enginePower', 'fuelType', 'mileage', 'model_name', 'name', 'sell_id', 'vehicleTransmission', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'years']


In [62]:
train_preproc = train_preproc[['bodyType', 'brand', 'color', 'description', 'engineDisplacement', 'enginePower', 'fuelType', 'mileage', 'model_name', 'name', 'sell_id', 'vehicleTransmission', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'years', 'price']]

In [42]:
display(train_preproc.describe())
display(train_preproc.describe(include = ['object']))

,description,mileage,Владельцы,sell_id,price,engineDisplacement,enginePower,years
count,26328.000000,26328.000000,26328.000000,2.632800e+04,2.632800e+04,26328.000000,26328.000000,26328.000000
mean,0.971209,150266.267510,2.102780,1.100427e+09,1.468780e+06,2.055150,200.027955,10.437747
std,0.167221,99483.303251,0.850639,1.588225e+07,1.660624e+06,1.015509,133.363129,7.122994
min,0.000000,1.000000,1.000000,2.665000e+03,3.000000e+04,0.000000,10.000000,0.000000
25%,1.000000,77737.500000,1.000000,1.101671e+09,5.550000e+05,1.000000,115.000000,5.000000
50%,1.000000,135454.000000,2.000000,1.102306e+09,9.900000e+05,2.000000,167.000000,9.000000
75%,1.000000,201000.000000,3.000000,1.102541e+09,1.797250e+06,3.000000,249.000000,14.000000
max,1.000000,1000000.000000,3.000000,1.102676e+09,5.250000e+07,6.000000,850.000000,84.000000


,name,brand,model_name,color,bodyType,vehicleTransmission,Привод,Руль,ПТС,fuelType
count,26328,26328,26328,26328,26328,26328,26328,26328,26328,26328
unique,2471,12,518,16,24,4,3,2,2,5
top,1.6 AT (110 л.с.),BMW,Octavia,чёрный,внедорожник 5 дв.,автоматическая,полный,Левый,Оригинал,Бензин
freq,581,3660,1038,8300,9842,15112,11645,25405,22926,21170


In [43]:
display(X_sub.describe())
display(X_sub.describe(include = ['object']))

,description,engineDisplacement,enginePower,mileage,sell_id,Владельцы,years
count,34686.0,34686.000000,34686.000000,34686.000000,3.468600e+04,34686.000000,34686.000000
mean,1.0,2.049617,197.818572,162009.767889,1.098300e+09,2.187338,11.735398
std,0.0,1.021544,133.299826,100676.559489,1.911225e+07,0.835266,7.047661
min,1.0,0.000000,10.000000,1.000000,2.665000e+03,1.000000,1.000000
25%,1.0,1.000000,110.000000,91153.500000,1.099049e+09,1.000000,7.000000
50%,1.0,2.000000,160.000000,149779.500000,1.100911e+09,2.000000,10.000000
75%,1.0,3.000000,240.000000,215000.000000,1.101245e+09,3.000000,15.000000
max,1.0,6.000000,850.000000,1000000.000000,1.101375e+09,3.000000,117.000000


,bodyType,brand,color,fuelType,model_name,name,vehicleTransmission,ПТС,Привод,Руль
count,34686,34686,34686,34686,34686,34686,34686,34686,34686,34686
unique,24,12,16,5,544,2780,4,2,3,2
top,седан,BMW,чёрный,бензин,OCTAVIA,1.6 AT (110 л.с.),автоматическая,Оригинал,передний,Левый
freq,13147,4473,11010,28601,1418,631,19596,30099,15565,33287


In [65]:
X = train_preproc.drop(['price'], axis=1,)
y = train_preproc.price.values

In [66]:
X_sub.nunique()

bodyType                  24
brand                     12
color                     16
description                1
engineDisplacement         7
enginePower              321
fuelType                   5
mileage                11268
model_name               544
name                    2780
sell_id                34686
vehicleTransmission        4
Владельцы                  3
ПТС                        2
Привод                     3
Руль                       2
years                     69
dtype: int64

In [67]:
VAL_SIZE   = 0.33   # 33%
N_FOLDS    = 5

# CATBOOST
ITERATIONS = 2000
LR         = 0.1
#Train Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [68]:
# чтобы не писать весь список этих признаков, просто вывел их через nunique(). и так сойдет)
X_train.nunique()

bodyType                  23
brand                     12
color                     16
description                2
engineDisplacement         7
enginePower              303
fuelType                   5
mileage                 6994
model_name               470
name                    2138
sell_id                17576
vehicleTransmission        4
Владельцы                  3
ПТС                        2
Привод                     3
Руль                       2
years                     63
dtype: int64

In [69]:
# Keep list of all categorical features in dataset to specify this for CatBoost
cat_features_ids = np.where(X_train.apply(pd.Series.nunique) < 3000)[0].tolist()

In [70]:
#Fit
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
model.fit(X_train, y_train,
         cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.5095005	test: 1.5374323	best: 1.5374323 (0)	total: 107ms	remaining: 3m 34s
100:	learn: 0.2192674	test: 0.2110404	best: 0.2110404 (100)	total: 5.64s	remaining: 1m 45s
200:	learn: 0.1968941	test: 0.1946626	best: 0.1946626 (200)	total: 11.3s	remaining: 1m 40s
300:	learn: 0.1874954	test: 0.1881169	best: 0.1881169 (300)	total: 16.9s	remaining: 1m 35s
400:	learn: 0.1796456	test: 0.1829007	best: 0.1828725 (391)	total: 22.8s	remaining: 1m 30s
500:	learn: 0.1734060	test: 0.1788121	best: 0.1784573 (492)	total: 28.7s	remaining: 1m 25s
600:	learn: 0.1694198	test: 0.1762164	best: 0.1762164 (600)	total: 34.8s	remaining: 1m 20s
700:	learn: 0.1647416	test: 0.1738652	best: 0.1738652 (700)	total: 41s	remaining: 1m 15s
800:	learn: 0.1615426	test: 0.1721122	best: 0.1721122 (800)	total: 47.2s	remaining: 1m 10s
900:	learn: 0.1580742	test: 0.1709409	best: 0.1709409 (900)	total: 53.4s	remaining: 1m 5s
1000:	learn: 0.1547365	test: 0.1696844	best: 0.1696844 (1000)	total: 59.9s	remaining: 59.8s
1100:

In [71]:
#Submission
predict_submission = model.predict(X_sub)
predict_submission

array([ 960668.33155452, 1274386.87762833,  883361.98993779, ...,
        186773.02733813, 1702510.11274719, 1296470.34432085])

In [75]:
sample_submission = pd.read_csv('drive/My Drive/Colab Notebooks/Project_6/sample_submission.csv')

In [77]:
VERSION    = 1
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'drive/My Drive/Colab Notebooks/Project_6/submission_v{VERSION}.csv', index=False)#r'drive/My Drive/Colab Notebooks/Project_6/
sample_submission.head(10)

,sell_id,price
0,1100575026,9.606683e+05
1,1100549428,1.274387e+06
2,1100658222,8.833620e+05
3,1100937408,8.172433e+05
4,1101037972,8.560236e+05
5,1100912634,1.052044e+06
6,1101228730,1.033642e+06
7,1100165896,6.909873e+05
8,1100768262,3.159357e+06
9,1101218501,1.131658e+06


In [78]:
def cat_model(y_train, X_train, X_test, y_test):
    model = CatBoostRegressor(iterations = ITERATIONS,
                              learning_rate = LR,
                              eval_metric='MAPE',
                              random_seed = RANDOM_SEED,)
    model.fit(X_train, y_train,
              cat_features=cat_features_ids,
              eval_set=(X_test, y_test),
              verbose=False,
              use_best_model=True,
              plot=False)
    
    return(model)


def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [79]:
submissions = pd.DataFrame(0,columns=["sub_1"], index=sample_submission.index) # куда пишем предикты по каждой модели
score_ls = []
splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))

for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=N_FOLDS,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # model for this fold
    model = cat_model(y_train, X_train, X_test, y_test,)
    # score model on test
    test_predict = model.predict(X_test)
    test_score = mape(y_test, test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {mape(y_test, test_predict):0.3f}")
    # submissions
    submissions[f'sub_{idx+1}'] = model.predict(X_sub)
    model.save_model(f'catboost_fold_{idx+1}.model')
    
print(f'Mean Score: {np.mean(score_ls):0.3f}')
print(f'Std Score: {np.std(score_ls):0.4f}')
print(f'Max Score: {np.max(score_ls):0.3f}')
print(f'Min Score: {np.min(score_ls):0.3f}')

1 Fold Test MAPE: 0.160
2 Fold Test MAPE: 0.166
3 Fold Test MAPE: 0.159
4 Fold Test MAPE: 0.164
5 Fold Test MAPE: 0.157

Mean Score: 0.161
Std Score: 0.0033
Max Score: 0.166
Min Score: 0.157
